In [2]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [3]:
import folium
import pandas as pd

In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

In [6]:
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

,Launch Site,Lat,Long
0,CCAFS LC-40,28.562302,-80.577356
1,CCAFS SLC-40,28.563197,-80.576820
2,KSC LC-39A,28.573255,-80.646895
3,VAFB SLC-4E,34.632834,-120.610745


In [7]:
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

In [8]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

In [9]:
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

In [14]:
# Task 2: Mark the success/failed launches for each site on the map

marker_cluster = MarkerCluster().add_to(site_map)

for index, row in spacex_df.iterrows():
    color = 'green' if row['class'] == 1 else 'red'
    label = 'Success' if row['class'] == 1 else 'Failure'

    folium.Marker(
        location=[row['Lat'], row['Long']],
        popup=f"{row['Launch Site']} - {label}",
        icon=folium.Icon(color=color)
    ).add_to(marker_cluster)
site_map

In [9]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [11]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

In [13]:
# Coordinates of the launch site (CCAFS SLC 40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# Coordinates of nearest coastline (from MousePosition plugin)
coastline_lat = 28.56367
coastline_lon = -80.57163

# Calculate the distance
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

# Display the distance
print(f"Distance from launch site to coastline: {distance:.2f} km")

Distance from launch site to coastline: 0.58 km


In [14]:
# 3. Add a marker for the coastline point
folium.Marker(
    location=[coastline_lat, coastline_lon],
    popup='Closest Coastline',
    icon=folium.Icon(color='cadetblue', icon='tint', prefix='fa')
).add_to(site_map)

# 4. Add a line from launch site to coastline
folium.PolyLine([[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
                color='blue', weight=2.5, tooltip='Distance Line').add_to(site_map)

# 5. Add a floating text marker showing distance
distance_marker = folium.Marker(
    location=[(launch_site_lat + coastline_lat) / 2, (launch_site_lon + coastline_lon) / 2],  # midpoint
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:#d35400;"><b>{:.2f} KM</b></div>'.format(distance),
    )
)
site_map.add_child(distance_marker)

# 6. Show the updated map
site_map

In [16]:
railway_lat = 28.56489
railway_lon = -80.58972
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)

In [17]:
# Marker for railway
folium.Marker(
    [railway_lat, railway_lon],
    popup='Railway Station',
    icon=folium.Icon(color='black', icon='train', prefix='fa')
).add_to(site_map)

# Line from launch site to railway
folium.PolyLine([[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
                color='purple', weight=2.5, tooltip='Distance to Railway').add_to(site_map)

# Midpoint text showing distance
mid_lat = (launch_site_lat + railway_lat) / 2
mid_lon = (launch_site_lon + railway_lon) / 2

folium.Marker(
    location=[mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 12px; color:purple;"><b>{:.2f} KM</b></div>'.format(distance_railway)
    )
).add_to(site_map)
site_map

In [25]:
highway_lat = 28.56301
highway_lon = -80.57085
# Calculate distance to highway
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)

# Marker for highway
folium.Marker(
    [highway_lat, highway_lon],
    popup='Nearest Highway',
    icon=folium.Icon(color='orange', icon='road', prefix='fa')
).add_to(site_map)

# Line to highway
folium.PolyLine([[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
                color='orange', weight=2.5, tooltip='Distance to Highway').add_to(site_map)

# Floating text label (distance)
mid_lat = (launch_site_lat + highway_lat) / 2
mid_lon = (launch_site_lon + highway_lon) / 2

folium.Marker(
    location=[mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:orange;"><b>{:.2f} KM</b></div>'.format(distance_highway)
    )
).add_to(site_map)
site_map

In [20]:
# Coordinates for the nearby city (e.g., Cocoa, FL)
city_lat = 28.386115
city_lon = -80.741998

# Calculate distance to city
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)

# ✅ Print the distance in output
print(f"Distance from Launch Site to Nearest City: {distance_city:.2f} KM")

# 🏙️ Marker for city
folium.Marker(
    location=[city_lat, city_lon],
    popup='Nearest City: Cocoa',
    icon=folium.Icon(color='darkpurple', icon='building', prefix='fa')
).add_to(site_map)

# ➖ Line to city
folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
    color='darkpurple',
    weight=2.5,
    tooltip='Distance to City'
).add_to(site_map)

# 📏 Distance label on map
mid_lat = (launch_site_lat + city_lat) / 2
mid_lon = (launch_site_lon + city_lon) / 2

folium.Marker(
    location=[mid_lat, mid_lon],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12px; color:darkpurple;"><b>{:.2f} KM</b></div>'.format(distance_city)
    )
).add_to(site_map)

# Show map
site_map

Distance from Launch Site to Nearest City: 25.36 KM


In [21]:
# Coordinates for Launch Site (e.g., CCAFS SLC 40)
launch_site_lat = 28.562302
launch_site_lon = -80.577356

# ----------------------------------------
# 1. Coastline
coastline_lat = 28.56367
coastline_lon = -80.57163
distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
print(f"Distance to Coastline: {distance_coastline:.2f} KM")

folium.Marker(
    [coastline_lat, coastline_lon],
    popup='Coastline',
    icon=folium.Icon(color='cadetblue', icon='tint', prefix='fa')
).add_to(site_map)

folium.PolyLine([[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
                color='blue', weight=2.5, tooltip='Distance to Coastline').add_to(site_map)

folium.Marker(
    location=[(launch_site_lat + coastline_lat)/2, (launch_site_lon + coastline_lon)/2],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:blue;"><b>{distance_coastline:.2f} KM</b></div>'
    )
).add_to(site_map)

# ----------------------------------------
# 2. Railway
railway_lat = 28.56489
railway_lon = -80.58972
distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
print(f"Distance to Railway: {distance_railway:.2f} KM")

folium.Marker(
    [railway_lat, railway_lon],
    popup='Railway Station',
    icon=folium.Icon(color='black', icon='train', prefix='fa')
).add_to(site_map)

folium.PolyLine([[launch_site_lat, launch_site_lon], [railway_lat, railway_lon]],
                color='purple', weight=2.5, tooltip='Distance to Railway').add_to(site_map)

folium.Marker(
    location=[(launch_site_lat + railway_lat)/2, (launch_site_lon + railway_lon)/2],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:purple;"><b>{distance_railway:.2f} KM</b></div>'
    )
).add_to(site_map)

# ----------------------------------------
# 3. Highway
highway_lat = 28.56301
highway_lon = -80.57085
distance_highway = calculate_distance(launch_site_lat, launch_site_lon, highway_lat, highway_lon)
print(f"Distance to Highway: {distance_highway:.2f} KM")

folium.Marker(
    [highway_lat, highway_lon],
    popup='Highway',
    icon=folium.Icon(color='orange', icon='road', prefix='fa')
).add_to(site_map)

folium.PolyLine([[launch_site_lat, launch_site_lon], [highway_lat, highway_lon]],
                color='orange', weight=2.5, tooltip='Distance to Highway').add_to(site_map)

folium.Marker(
    location=[(launch_site_lat + highway_lat)/2, (launch_site_lon + highway_lon)/2],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:orange;"><b>{distance_highway:.2f} KM</b></div>'
    )
).add_to(site_map)

# ----------------------------------------
# 4. City
city_lat = 28.386115
city_lon = -80.741998
distance_city = calculate_distance(launch_site_lat, launch_site_lon, city_lat, city_lon)
print(f"Distance to City: {distance_city:.2f} KM")

folium.Marker(
    [city_lat, city_lon],
    popup='City: Cocoa',
    icon=folium.Icon(color='darkpurple', icon='building', prefix='fa')
).add_to(site_map)

folium.PolyLine([[launch_site_lat, launch_site_lon], [city_lat, city_lon]],
                color='darkpurple', weight=2.5, tooltip='Distance to City').add_to(site_map)

folium.Marker(
    location=[(launch_site_lat + city_lat)/2, (launch_site_lon + city_lon)/2],
    icon=DivIcon(
        icon_size=(150, 36),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:darkpurple;"><b>{distance_city:.2f} KM</b></div>'
    )
).add_to(site_map)

# ----------------------------------------
# Display the map
site_map

Distance to Coastline: 0.58 KM
Distance to Railway: 1.24 KM
Distance to Highway: 0.64 KM
Distance to City: 25.36 KM
